In [2]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files
import requests # library to handle requests
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors



# import k-means from clustering stage
from sklearn.cluster import KMeans


!conda install -c conda-forge folium=0.5.0 --yes
#import folium # map rendering library
!pip install folium
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |   py36h9f0ad1d_1         157 KB  conda-forge
    soupsieve-1.9.4            |   py36h9f0ad1d_1          58 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 M

Getting the required table

In [10]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#url = ('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695')
#results = requests.get(url).json
#results

In [11]:
soup = BeautifulSoup(data, 'html.parser')

In [12]:
PostalCodeList1 = []
BoroughList1 = []
NeighborhoodList1 = []

In [13]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [14]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodeList1.append(cells[0].text.rstrip('\n'))
        BoroughList1.append(cells[1].text.rstrip('\n'))
        NeighborhoodList1.append(cells[2].text.rstrip('\n'))

In [15]:
toronto_df1 = pd.DataFrame({"PostalCode": PostalCodeList1,
                           "Borough": BoroughList1,
                           "Neighborhood": NeighborhoodList1})

toronto_df1.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode           Borough                Neighborhood
0        M1A      Not assigned                            
1        M2A      Not assigned                            
2        M3A        North York                   Parkwoods
3        M4A        North York            Victoria Village
4        M5A  Downtown Toronto  Regent Park / Harbourfront

In [16]:
toronto_df_drop1 = toronto_df1[toronto_df1.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop1.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode           Borough                                  Neighborhood
0        M3A        North York                                     Parkwoods
1        M4A        North York                              Victoria Village
2        M5A  Downtown Toronto                    Regent Park / Harbourfront
3        M6A        North York             Lawrence Manor / Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government

In [17]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_drop1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode      Borough                              Neighborhood
0        M1B  Scarborough                           Malvern / Rouge
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek
2        M1E  Scarborough       Guildwood / Morningside / West Hill
3        M1G  Scarborough                                    Woburn
4        M1H  Scarborough                                 Cedarbrae

In [18]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode      Borough                              Neighborhood
0        M1B  Scarborough                           Malvern / Rouge
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek
2        M1E  Scarborough       Guildwood / Morningside / West Hill
3        M1G  Scarborough                                    Woburn
4        M1H  Scarborough                                 Cedarbrae

In [19]:
toronto_df_grouped.shape

(103, 3)

In [20]:
#Load Geo Data from CVS
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [21]:
toronto_df_geo = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_geo.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode      Borough                              Neighborhood  \
0        M1B  Scarborough                           Malvern / Rouge   
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2        M1E  Scarborough       Guildwood / Morningside / West Hill   
3        M1G  Scarborough                                    Woburn   
4        M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  

In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df_geo['Latitude'], toronto_df_geo['Longitude'], toronto_df_geo['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [24]:
CLIENT_ID = '05Y4VHYPLZQYH2NXVX5CDSII0FNABH54BRFIFL02GNRCXP0N' # your Foursquare ID
CLIENT_SECRET = 'RJMXBRLBG2K0VKK0RZEBDQYYKSYUEY4LJUCLSLBRUIZQL3Y3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 05Y4VHYPLZQYH2NXVX5CDSII0FNABH54BRFIFL02GNRCXP0N
CLIENT_SECRET:RJMXBRLBG2K0VKK0RZEBDQYYKSYUEY4LJUCLSLBRUIZQL3Y3


### Lets explore 3rd neighborhood

In [26]:
toronto_df_geo.loc[2, 'Neighborhood']

'Guildwood / Morningside / West Hill'

In [27]:
neighborhood_latitude = toronto_df_geo.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df_geo.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df_geo.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Guildwood / Morningside / West Hill are 43.7635726, -79.1887115.


### Generate the url in order to make a searching in 500 radius

In [28]:
LIMIT = 100 
radius = 500
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url1

'https://api.foursquare.com/v2/venues/explore?&client_id=05Y4VHYPLZQYH2NXVX5CDSII0FNABH54BRFIFL02GNRCXP0N&client_secret=RJMXBRLBG2K0VKK0RZEBDQYYKSYUEY4LJUCLSLBRUIZQL3Y3&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100'

### And Venues in this Neighborhood in 500 m radius are:

In [29]:
results = requests.get(url1).json()
results

{'meta': {'code': 200, 'requestId': '5e8852ce0cc1fd001bcdb31b'},
 'response': {'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c62f34bde1b2d7fec89e370',
       'name': 'G & G Electronics',
       'location': {'address': '4371 kingston road',
        'lat': 43.765309,
        'lng': -79.191537,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.765309,
          'lng': -79.191537}],
        'distance': 298,
        'postalCode': 'M1E 2M9',
       

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: module 'pandas.compat' has no attribute 'string_types'

In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Nearby avenues are :

In [34]:
toronto_venues = getNearbyVenues(names=toronto_df_geo['Neighborhood'],
                                   latitudes=toronto_df_geo['Latitude'],
                                   longitudes=toronto_df_geo['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(2202, 7)


ImportError: cannot import name 'OrderedDict'

                               Neighborhood  Neighborhood Latitude  \
0                           Malvern / Rouge              43.806686   
1  Rouge Hill / Port Union / Highland Creek              43.784535   
2  Rouge Hill / Port Union / Highland Creek              43.784535   
3       Guildwood / Morningside / West Hill              43.763573   
4       Guildwood / Morningside / West Hill              43.763573   

   Neighborhood Longitude                           Venue  Venue Latitude  \
0              -79.194353                         Wendy’s       43.807448   
1              -79.160497           Royal Canadian Legion       43.782533   
2              -79.160497  Scarborough Historical Society       43.788755   
3              -79.188711               G & G Electronics       43.765309   
4              -79.188711                Big Bite Burrito       43.766299   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085            

In [36]:
toronto_venues.groupby('Neighborhood').count()

ImportError: cannot import name 'OrderedDict'

                                                   Neighborhood Latitude  \
Neighborhood                                                               
Agincourt                                                              4   
Alderwood / Long Branch                                               10   
Bathurst Manor / Wilson Heights / Downsview North                     19   
Bayview Village                                                        4   
Bedford Park / Lawrence Manor East                                    25   
...                                                                  ...   
Wexford / Maryvale                                                     4   
Willowdale                                                            41   
Woburn                                                                 4   
Woodbine Heights                                                      11   
York Mills West                                                        4   

           

In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

ImportError: cannot import name 'OrderedDict'

   Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                  0                  0        0                   0   
1                  0                  0        0                   0   
2                  0                  0        0                   0   
3                  0                  0        0                   0   
4                  0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  ...  Turkish Restaurant  \
0                    0             0  ...                   0   
1               

In [39]:
toronto_onehot.shape

(2202, 268)

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

ImportError: cannot import name 'OrderedDict'

                                         Neighborhood  Accessories Store  \
0                                           Agincourt                0.0   
1                             Alderwood / Long Branch                0.0   
2   Bathurst Manor / Wilson Heights / Downsview North                0.0   
3                                     Bayview Village                0.0   
4                  Bedford Park / Lawrence Manor East                0.0   
..                                                ...                ...   
89                                 Wexford / Maryvale                0.0   
90                                         Willowdale                0.0   
91                                             Woburn                0.0   
92                                   Woodbine Heights                0.0   
93                                    York Mills West                0.0   

    Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                 0

In [41]:
toronto_grouped.shape

(94, 268)

#### Let's print each neighborhood along with the top 2 most common venues

In [43]:
num_top_venues = 2

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1  Latin American Restaurant  0.25


----Alderwood / Long Branch----
         venue  freq
0  Pizza Place   0.2
1          Pub   0.1


----Bathurst Manor / Wilson Heights / Downsview North----
         venue  freq
0         Bank  0.11
1  Coffee Shop  0.11


----Bayview Village----
                venue  freq
0                Bank  0.25
1  Chinese Restaurant  0.25


----Bedford Park / Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1         Pizza Place  0.08


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05


----Birch Cliff / Cliffside West----
             venue  freq
0             Café  0.25
1  College Stadium  0.25


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09


----Business reply mail Processing CentrE----
           venue  freq
0    Yoga Studio

#### Let's put that into a *pandas* dataframe

In [44]:
#Rows in decdeing order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

ImportError: cannot import name 'OrderedDict'

                                        Neighborhood  \
0                                          Agincourt   
1                            Alderwood / Long Branch   
2  Bathurst Manor / Wilson Heights / Downsview North   
3                                    Bayview Village   
4                 Bedford Park / Lawrence Manor East   

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  
0  Latin American Restaurant          Skating Rink        Breakfast Spot  
1                Pizza Place          Dance Studio              Pharmacy  
2                       Bank           Coffee Shop        Ice Cream Shop  
3                       Café                  Bank    Chinese Restaurant  
4                 Restaurant        Sandwich Place    Italian Restaurant  

## Cluster Neighborhoods

In [47]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10].astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [48]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df_geo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head # check the last columns!
toronto_merged.dtypes

PostalCode                object
Borough                   object
Neighborhood              object
Latitude                 float64
Longitude                float64
Cluster Labels           float64
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
dtype: object

### Finally, let's visualize the resulting clusters

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #I could not get color or fill color to work because there are neighborhoods with no top venues due to virus, and I cant covert ClusterLables to in wiht Nulls
        color = 'blue',
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters